# Subgraph Matching
## This notebook illustrates how you can find out if one graph is a subgraph of another supergraph. And if so, it will return the list of possible matches of the subgraph within the supergraph.

In [ ]:
# Import TopologicPy modules. This is not needed on other computers
import sys
sys.path.append("C:/Users/sarwj/OneDrive - Cardiff University/Documents/GitHub/topologicpy/src")

In [ ]:
from topologicpy.Edge import Edge
from topologicpy.CellComplex import CellComplex
from topologicpy.Graph import Graph
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Helper import Helper
print("This jupyter notebook requires topologicpy v.0.8.20 or newer")
print(Helper.Version())

In [ ]:
c = CellComplex.Prism(width=2, length=2, height=2)
cells = Topology.Cells(c)
for i, cell in enumerate(cells):
    d = Dictionary.ByKeyValue("id", "c_"+str(i+1))
    cell = Topology.SetDictionary(cell, d)
faces = Topology.Faces(c)
for i, face in enumerate(faces):
    d = Dictionary.ByKeyValue("id", "f_"+str(i+1))
    face = Topology.SetDictionary(face, d)
supergraph = Graph.ByTopology(c, toExteriorTopologies=True)
vertices = Graph.Vertices(supergraph)
edges = Graph.Edges(supergraph)

subgraph = Graph.ByVerticesEdges(vertices, edges[:-10])
subgraph = Graph.RemoveIsolatedVertices(subgraph)


# Modify super graph by adding intermediate vertices
new_edges = []
for i, edge in enumerate(edges):
    sv = Edge.StartVertex(edge)
    c = Topology.Centroid(edge)
    c = Topology.Translate(c, 0, 0, 0.1)
    d = Dictionary.ByKeyValue("id", "n_"+str(i+1))
    c = Topology.SetDictionary(c, d)
    ev = Edge.EndVertex(edge)
    e1 = Edge.ByVertices(sv, c)
    e2 = Edge.ByVertices(c, ev)
    new_edges.append(e1)
    new_edges.append(e2)
    vertices.append(c)
supergraph = Graph.ByVerticesEdges(vertices, new_edges)
# Add more edges to complete the supergraph
#supergraph = Graph.Complete(supergraph)

print("The supergraph")
Topology.Show(supergraph, vertexSize=12, vertexLabelKey="id", showVertexLabel=True)
print("The subgraph")
Topology.Show(Graph.Reshape(subgraph), vertexSize=12, vertexLabelKey="id", showVertexLabel=True)


In [ ]:
# Set the display attributes of the supergraph
vertices = Graph.Vertices(supergraph)
for v in vertices:
    d = Topology.Dictionary(v)
    d = Dictionary.SetValuesAtKeys(d, keys=["vertex_size", "vertex_color"], values=[8, "black"])
    v = Topology.SetDictionary(v, d)

# Get the list of matched subgraphs.
matched_subgraphs = Graph.SubGraphMatches(subgraph, supergraph, strict=False)

for i, matched_subgraph in enumerate(matched_subgraphs):
    edges = Graph.Edges(matched_subgraph)
    for e in edges:
        d = Dictionary.ByKeysValues(["edge_width", "edge_color"], [12, "blue"])
        e = Topology.SetDictionary(e, d)

    # Set the display attributes of the matched subgraph
    vertices = Graph.Vertices(matched_subgraph)
    for v in vertices:
        d = Topology.Dictionary(v)
        d = Dictionary.SetValuesAtKeys(d, keys=["vertex_size", "vertex_color"], values=[16, "blue"])
        v = Topology.SetDictionary(v, d)
    
    # Show the matched subgraph superimposed over the supergraph
    print(str(i+1)+". Subgraph match superimposed over the supergaph")
    Topology.Show(supergraph, matched_subgraph, vertexSize=8, vertexLabelKey="id", showVertexLabel=True, vertexSizeKey="vertex_size", vertexColorKey="vertex_color", edgeWidthKey="edge_width", edgeColorKey="edge_color")



In [ ]:
g_top = Graph.Topology(supergraph)
g_top = Topology.Scale(g_top, Topology.Centroid(g_top), 1.1, 1.1, 1.1)
verts = Topology.Vertices(g_top)
edges = Topology.Edges(g_top)
g2 = Graph.ByVerticesEdges(verts, edges)
similarity = Graph.Compare(supergraph, g2,
                           weightJaccard=1,
                           weightClosenessCentrality=1,
                           weightStructure=1,
                           vertexIDKey="id",
                           edgeWeightKey="length",
                           mantissa=2,
                           silent=False)
for key in similarity.keys():
    print(key+":", similarity[key])


In [ ]:


edges = Graph.Edges(g2)
for e in edges:
    d = Dictionary.ByKeysValues(["edge_width", "edge_color"], [12, "blue"])
    e = Topology.SetDictionary(e, d)

# Set the display attributes of the matched subgraph
vertices = Graph.Vertices(matched_subgraph)
for v in vertices:
    d = Topology.Dictionary(v)
    d = Dictionary.SetValuesAtKeys(d, keys=["vertex_size", "vertex_color"], values=[16, "blue"])
    v = Topology.SetDictionary(v, d)
    
    # Show the matched subgraph superimposed over the supergraph
    #print(str(i+1)+". Subgraph match superimposed over the supergaph")
Topology.Show(supergraph, g2, vertexSize=8, vertexLabelKey="id", showVertexLabel=True, vertexSizeKey="vertex_size", vertexColorKey="vertex_color", edgeWidthKey="edge_width", edgeColorKey="edge_color")
